# Ayudantía 1 - Sistemas Recomendadores: Pyreclab

En este práctico vamos a utilizar la biblioteca de Python [pyreclab](https://github.com/gasevi/pyreclab), desarrollado por los Laboratorios IALab y SocVis de la Pontificia Universidad Católica de Chile, para aprender sobre algoritmos básicos y tradicionales de sistemas de recomendacion:


*   Most Popular
*   Item Average Rating
*   User KNN (Filtrado colaborativo basado en usuarios)


**Autores**: Denis Parra, Gabriel Sepúlveda

**Adaptado para Sistemas Recomendadores (IIC3633) por**: Manuel Cartagena, Antonio Ossa


## Setup

Vaya ejecutando cada celda presionando el botón de Play o presionando Ctrl+Enter (Linux y Windows) o Command+Enter (OSX)

**Paso 1:** Descargue los siguientes tres archivos, siguiendo los links, a su computador local (`u1.base`, `u1.test` y `u.item`, respectivamente)

https://drive.google.com/file/d/1Anro2DyEgN0sUHXFyxVe2qoSya6hzQSp/view?usp=sharing

https://drive.google.com/file/d/1Awy3QKCdkSpNeRXATqRBUMi56M47Ia0n/view?usp=sharing

https://drive.google.com/file/d/1B7QhEodQ13QlOHOTp9zaa52exlfgAOl3/view?usp=sharing


In [0]:
# Ejecute esta celda. Deberá subir los archivos u1.base, u1.test y u.item
from google.colab import files
uploaded = files.upload()

Los archivos u1.base y u1.test tienen tuplas {usuario, item, rating, timestamp}, que es la información de preferencias de usuarios sobre películas en una muestra del dataset [MovieLens](https://grouplens.org/datasets/movielens/).

Revisemos cómo es uno de estos archivos:

In [0]:
import pandas as pd

train_file = pd.read_csv('u1.base',
                         sep='\t',
                         names = ['userid', 'itemid', 'rating', 'timestamp'],
                         header=None)
train_file.head()

Por otra parte, para obtener información adicional de cada película tal como *título*, *fecha de lanzamient*o, *género*, etc., cargaremos el archivo de items descargado ( *u.item* ) para poder mapear cada identificador de ítem al conjunto de datos que lo describe.
Revisemos el contenido de este archivo

In [0]:
info_cols = [ 'movieid', 'title', 'release_date', 'video_release_date', 'IMDb_URL', \
              'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', \
              'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', \
              'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western' ]

info_file = pd.read_csv('u.item', sep='|', index_col = 0, names = info_cols, header=None, encoding='latin-1')

info_file.head()

In [0]:
# Ejemplo de cómo visualizar titulos de peliculas en base a sus IDs
pelis = [5,4,1]
info_file.loc[pelis]['title']

**Paso 2:** Instalar `pyreclab`



In [0]:
!pip install pyreclab

**Paso 3:** Importar `pyreclab` y `numpy`

In [0]:
import pyreclab
import numpy as np

## Uso de algoritmos

### *Most Popular*

In [0]:
# Declarar el objeto recomendador de la clase MostPopular
mymp = pyreclab.MostPopular(dataset='u1.base',
                            dlmchar= b'\t',
                            header=False,
                            usercol=0,
                            itemcol=1,
                            ratingcol=2)


In [0]:
# Entrenamos el modelo con los datos existentes
mymp.train()

El método ***Most Popular*** calcula el número de veces que fue consumido cada ítem, así puede recomendar los más populares. Con este método no podemos predecir *ratings*, pero sí podemos recomendar. La recomendación no es personalizada, es la misma para todos los usuarios.

In [0]:
reclist = mymp.recommend("457")
print(reclist)

In [0]:
# Con esto podemos ver las recomendaciones con titulo de película
pelis = np.array(reclist).astype(int)
info_file.loc[pelis]['title']

### *Item Average*

In [0]:
# Declarar el objeto recomendador ItemAvg
myitemavg = pyreclab.ItemAvg(dataset='u1.base',
                             dlmchar=b'\t',
                             header=False,
                             usercol=0,
                             itemcol=1,
                             ratingcol=2)


In [0]:
# Entrenamos el modelo con los datos existentes
myitemavg.train()

#### Preguntas

**¿Qué cree usted que hace la función `train()` del método de recomendación  `ItemAvg()`?**

**Respuesta:** COMPLETAR

In [0]:
# Predecir rating que el usuario ID 457 le dará al ítem ID 37
myitemavg.predict('457', '37')

**Ejecute el mismo comando anterior para predecir esta vez el rating que el usuario 498 dará al item 37 ¿Qué rating da? ¿Cómo se explica este resultado?**

In [0]:
# COMPLETAR

**Respuesta:** COMPLETAR

In [0]:
# Generar lista de 5 recomendaciones para el usuario con ID 457

reclist_iavg = myitemavg.recommend('457', 5)
print(reclist_iavg)

**Genere una lista de 10 recomendaciones para el usuario ID 478 ¿Cómo se explican las recomendaciones del método para este usuario comparadas con las del usuario 457?**

In [0]:
# COMPLETAR

**Respuesta:** COMPLETAR

**¿Cuáles son los títulos de las películas recomendadas por el método *ItemAvg* para el usuario 457 ? ¿Qué le parecen estas recomendaciones comparadas con el método *MostPopular*?**

In [0]:
# COMPLETAR

**Respuesta:** COMPLETAR

### Wilson score

Otra forma de recomendar cuando se tienen votos positivos y negativos es usando el lower bound del wilson-score como se muestra en https://www.evanmiller.org/how-not-to-sort-by-average-rating.html

Como el dataset de movielens contiene ratings, una forma de contabilizar estos como votos es asignando los ratings superiores a un valor definido como votos positivos y los demás como negativos.

In [0]:
from math import sqrt
import scipy.stats as st

def ci_lower_bound(pos, neg, confidence=.95):
    n = pos + neg

    if n == 0:
        return 0

    #z = 1.0 #1.44 = 85%, 1.96 = 95%
    z = st.norm.ppf(1-(1-confidence)/2)
    phat = float(pos) / n
    
    return (phat + z*z/(2*n) - z * sqrt((phat*(1-phat)+z*z/(4*n))/n))/(1+z*z/n)

In [0]:
item_count = {}
for tup in train_file.groupby(['itemid', 'rating']).agg(['count']).itertuples():
  if tup[0][0] not in item_count:
    item_count[tup[0][0]] = {}
    item_count[tup[0][0]]['pos'] = 0
    item_count[tup[0][0]]['neg'] = 0

  if tup[0][1] <= 3:
    item_count[tup[0][0]]['neg'] += tup[1]
  else:
    item_count[tup[0][0]]['pos'] += tup[1]

A partir de la funcion ci_lower_bound y el diccionario item_count genere una lista de recomendación usando el wilson score.

PREGUNTAS:
1. Cómo considera este ranking respecto a Most Popular y a Rating Promedio? explique.

2. Cuál sería el efecto en el ranking de cambiar el umbral de rating positivo de  $r \geq 4$ a $r \geq 3$

3. Cuál es el efecto de cambiar el valor (1-$\alpha$) de 0.95 a 0.99 en la lista rankeada ?

### *UserKNN*

In [0]:
# Declarar el objeto recomendador UserKnn
myUserKnn = pyreclab.UserKnn(dataset='u1.base',
                             dlmchar=b'\t',
                             header=False,
                             usercol=0,
                             itemcol=1,
                             ratingcol=2)


In [0]:
# Entrenamos el modelo con los datos existentes
# Recuerde que se puede probar el parámetro k de cantidad de vecinos
# así como la métrica de similaridad (pearson, cosine)
myUserKnn.train(7, 'pearson')

#### Preguntas

**Según el modelo de recomendación UserKnn, qué rating le dará el usuario 457 al item 37 ?**

In [0]:
# COMPLETAR

**Respuesta:** COMPLETAR

**Escriba los nombres de las películas recomendadas por el método knn**

In [0]:
# COMPLETAR

**Respuesta:** COMPLETAR

Analizar de tal forma de minimizar el número de vecinos viendo el error (RMSE)

Un ejemplo de como obtener las listas de recomendación junto con sus métricas para el dataset de test:

In [0]:
predlist, mae, rmse = myUserKnn.test(input_file='u1.test',
                                     dlmchar=b'\t',
                                     header=False,
                                     usercol=0,
                                     itemcol=1,
                                     ratingcol=2,
                                     output_file='predictions.csv')

A partir del ejemplo anterior probar para distintos valores de k y evaluar en base al error de prediccion (RMSE)

In [0]:
#COMPLETAR

Graficar los resultados anteriores (MAE y RMSE), los que deberían verse algo así:

![MAE](https://fotos.subefotos.com/819d05796c6e1da8ec11637788c677aco.png)

![RMSE](https://fotos.subefotos.com/6148bcf1c0fdd7ffc0046b04f3f455b5o.png)

1.   Elemento de lista
2.   Elemento de lista



### *Slope One* (Opcional)

**Replicar todo el análisis de UserKNN con el método SlopeOne
**

### *ItemKNN* (Opcional)

**Replicar todo el análisis de UserKNN con el método ItemKNN**